In [ ]:
#!pip install --upgrade tensorflow

In [24]:
from google.colab import drive
drive.mount('/content/gdrive')
cd '/content/gdrive/My Drive/AllDeepLearning'

Mounted at /content/gdrive


In [27]:
#!unzip -qq "DeepLearningZeroToAll.zip"

In [61]:
import tensorflow.compat.v1 as tf

In [9]:
tf.__version__

'2.5.0'

In [11]:
hello = tf.constant("Hello, Tensorflow!") #하나의 노드를 만드는 것
sess = tf.Session() #세션을 만들기

print(sess.run(hello))#세션 만들어 노드를 실행

AttributeError: ignored

### version 2.xx 에서는 Session 정의 과정이 생략됨 

In [12]:
hello = tf.constant("Hello, Tensorflow!")
tf.print(hello)

Hello, Tensorflow!


## Computational Graph

with tf.Session() as sess: 를 추가해야 에러 안뜸

In [39]:
#Build graph

with tf.Session() as sess:
  node1 = tf.constant(3.0, tf.float32)
  node2 = tf.constant(4.0) # also tf.float32 implicitly
  node3 = tf.add(node1, node2)


In [40]:
print("node1:", node1, "node2:", node2)
print("node3: ", node3)

node1: Tensor("Const:0", shape=(), dtype=float32) node2: Tensor("Const_1:0", shape=(), dtype=float32)
node3:  Tensor("Add:0", shape=(), dtype=float32)


In [41]:
sess = tf.Session()
print("sess.run(node1, node2): ", sess.run([node1, node2]))
print("sess.run(node3): ", sess.run(node3))

sess.run(node1, node2):  [3.0, 4.0]
sess.run(node3):  7.0


Placeholder라고 특별한 노드로 그래프를 만들면 미리 값 정하지 않고 


run 시킬 때 값을 줘서 실행시킬 수 있음

In [44]:
tf.compat.v1.disable_eager_execution() #version2라서 이거 추가해줘야됨

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)


with tf.Session() as sess:
  print(sess.run(adder_node, feed_dict={a: 3, b: 4.5}))
  print(sess.run(adder_node, feed_dict={a: [1,3], b: [2, 4]}))

7.5
[3. 7.]


### TensorFlow Mechanics

1. Build Graph using TensorFlow operations
2. feed data and run graph(operation)
  <br> sess.run(op, feed_dict={x:x_data})
3. update variables in the graph and return values

# Lec2

In [48]:
tf.set_random_seed(777)  # for reproducibility

# X and Y data
x_train = [1, 2, 3]
y_train = [1, 2, 3]

#### tr.Variable


우리가 구해야하는 값, 즉 trainable 값이다!


tensorflow가 학습하는 값 

<br>

#### tr.random_normal([~])
W,b의 초기값으로 random한 값으로 부여


W,b의 차원은 1이라서 아래서 1로 정의


In [49]:
W = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Our hypothesis XW+b
hypothesis = x_train * W + b

In [51]:
# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - y_train))

In [52]:
# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

#그래프 구축 완료

In [ ]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer()) #variable 실행 전에 요거 실행시켜줘야됨

# Fit the line
for step in range(2001):
    sess.run(train) #train 노드를 실행시킨다
    if step % 20 == 0:
        print(step, sess.run(cost), sess.run(W), sess.run(b))
        

In [58]:
print(sess.run(train)) #암것두 안뜸

None


#### Placeholder 사용해서 구현해보기
위에서는 처음에 x_train = [1,2,3],  y_train~~ 정의한 후
hypothesis = x_train*W +b 해서 그래프를 구축

Placeholder 에서는 sess.run([cost, W, b, train], feed_dict={X: [1, 2, 3], Y: [1, 2, 3]}) 

이렇게 session을 실행할 때 정의

In [63]:
tf.set_random_seed(777)  # for reproducibility

# Try to find values for W and b to compute y_data = W * x_data + b
# We know that W should be 1 and b should be 0
# But let's use TensorFlow to figure it out
W = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Now we can use X and Y in place of x_data and y_data
# # placeholders for a tensor that will be always fed using feed_dict
# See http://stackoverflow.com/questions/36693740/
X = tf.placeholder(tf.float32, shape=[None]) #shape이 None이면 아무값 다 된다는
Y = tf.placeholder(tf.float32, shape=[None])


In [64]:

# Our hypothesis XW+b
hypothesis = X * W + b

# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())


In [ ]:

# Fit the line
for step in range(2001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train], #이렇게 한번에 run 시킬수도 있음
                 feed_dict={X: [1, 2, 3], Y: [1, 2, 3]}) 
    if step % 20 == 0:
        print(step, cost_val, W_val, b_val)


#### Testing our model

In [ ]:


print(sess.run(hypothesis, feed_dict={X: [5]}))
print(sess.run(hypothesis, feed_dict={X: [2.5]}))
print(sess.run(hypothesis, feed_dict={X: [1.5, 3.5]}))


In [ ]:

# Fit the line with new training data
for step in range(2001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train],
                 feed_dict={X: [1, 2, 3, 4, 5],
                            Y: [2.1, 3.1, 4.1, 5.1, 6.1]})
    if step % 20 == 0:
        print(step, cost_val, W_val, b_val)

# Testing our model
print(sess.run(hypothesis, feed_dict={X: [5]}))
print(sess.run(hypothesis, feed_dict={X: [2.5]}))
print(sess.run(hypothesis, feed_dict={X: [1.5, 3.5]}))